In [1]:
import pandas as pd

import sys
sys.path.append('../helper_functions')
from helper_functions import impute_with

In [2]:
X_test = pd.read_csv('../project_data/test_data.csv', delimiter=',',dtype={'Zip Code': str})

pd.set_option('display.max_columns', None)

# save the original indexies of the test data
X_test_index = X_test.index

# drop the columns that have no information and are not getting use in the end 
columns_to_drop = ['Industry Code Description',
                   'OIICS Nature of Injury Description', 
                   'WCIO Cause of Injury Description', 
                   'WCIO Nature of Injury Description', 
                   'WCIO Part Of Body Description',
                   'Zip Code',
                   'Carrier Name',
                   'County of Injury',
                   'District Name']

X_test.drop(columns = columns_to_drop, inplace= True)

In [3]:
# read in the X_train data

X_train = pd.read_csv('../project_data/train_preprocessed.csv', delimiter=',',dtype={'Zip Code': str})

transform date variables into datetime with pandas 

In [4]:
# Identify original missing values
date_columns = ['Accident Date', 'Assembly Date', 'C-2 Date', 'C-3 Date', 'First Hearing Date']
original_missing_test = X_test[date_columns].isna()
original_missing_train = X_train[date_columns].isna()


# Convert columns to datetime
for column in date_columns:
    X_test[column] = pd.to_datetime(X_test[column], format='%Y-%m-%d', errors='coerce')

    # Find new NaT values that weren't originally missing
    invalid_dates = X_test[X_test[column].isna() & ~original_missing_test[column]]
    if not invalid_dates.empty:
        print(f"Invalid dates found in column '{column}':")
        print(invalid_dates[[column]])


# Convert columns to datetime
for column in date_columns:
    X_train[column] = pd.to_datetime(X_train[column], format='%Y-%m-%d', errors='coerce')

    # Find new NaT values that weren't originally missing
    invalid_dates = X_train[X_train[column].isna() & ~original_missing_train[column]]
    if not invalid_dates.empty:
        print(f"Invalid dates found in column '{column}':")
        print(invalid_dates[[column]])

In [5]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387975 entries, 0 to 387974
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Accident Date                   385531 non-null  datetime64[ns]
 1   Age at Injury                   387975 non-null  int64         
 2   Alternative Dispute Resolution  387975 non-null  object        
 3   Assembly Date                   387975 non-null  datetime64[ns]
 4   Attorney/Representative         387975 non-null  object        
 5   Average Weekly Wage             368771 non-null  float64       
 6   Birth Year                      368505 non-null  float64       
 7   C-2 Date                        378841 non-null  datetime64[ns]
 8   C-3 Date                        85216 non-null   datetime64[ns]
 9   Carrier Type                    387975 non-null  object        
 10  Claim Identifier                387975 non-null  int64  

In [6]:
# pd.concat([X_test.isna().sum()[X_test.isna().sum() > 0] ,X_test.isna().sum()[X_test.isna().sum() > 0] ])


# For the Test data we have to impute missing values 

# We also need to impute outliers and weird values that don't make sense


# Missing values are only 
pd.concat([X_test.isna().sum()[X_test.isna().sum() > 0] / X_test.shape[0], X_test.isna().sum()[X_test.isna().sum() > 0] ], axis = 1) # , names= ['rel_missing','abs_missing']




# For the Test data we have to impute missing values 
# Alternative Dispute Resolution         4
# C-2 Date                            9067 / we encode this with true and false
# C-3 Date                          270351 / we encode this wiht true and false 
# Carrier Type                        1195 / we have to impute this her e
# County of Injury                     804 / / we dont use this for our model 
# First Hearing Date                294696 / we encode this with a true and false 
# Gender                              3275 / we have to impute this here

 
# Accident Date                           2444 /
# Average Weekly Wage                    19204
# Birth Year                             19470
# C-2 Date                                9134
# C-3 Date                              302759
# First Hearing Date                    344947
# IME-4 Count                           352726
# Industry Code                           7736
# Industry Code Description               7736
# OIICS Nature of Injury Description    387975
# WCIO Cause of Injury Code              10348
# WCIO Cause of Injury Description       10348
# WCIO Nature of Injury Code             10560
# WCIO Nature of Injury Description      10560
# WCIO Part Of Body Code                  9549
# WCIO Part Of Body Description           9549
# Zip Code                               19342

,0,1
Accident Date,0.006299,2444
Average Weekly Wage,0.049498,19204
Birth Year,0.050184,19470
C-2 Date,0.023543,9134
C-3 Date,0.780357,302759
First Hearing Date,0.889096,344947
IME-4 Count,0.909146,352726
Industry Code,0.019939,7736
WCIO Cause of Injury Code,0.026672,10348
WCIO Nature of Injury Code,0.027218,10560


In [7]:
X_test['Industry Code'].unique()

array([48., 45., 56., 55., 72., 32., 44., 53., 92., 52., 61., 22., 33.,
       71., 81., 31., 62., 23., 54., 42., 51., 11., 49., 21., nan])

In [8]:
# check where rows have more than 60% missing values 
# Calculate the percentage of missing values in each row
missing_percentage = X_test.isna().mean(axis=1)

# Filter rows where the missing percentage is greater than 60%
rows_with_high_missing = X_test[missing_percentage > 0.035]

# Display rows with more than 60% missing values
rows_with_high_missing

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Type,Claim Identifier,COVID-19 Indicator,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents
0,2022-12-24,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaT,1A. PRIVATE,6165911,N,NaT,M,NaN,48.0,IV,31.0,10.0,54.0,1
1,2022-11-20,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaT,1A. PRIVATE,6166141,N,NaT,F,NaN,45.0,IV,75.0,10.0,10.0,1
2,2022-12-26,59,N,2023-01-02,N,0.0,1963.0,2022-12-31,NaT,1A. PRIVATE,6165907,N,NaT,F,NaN,56.0,III,68.0,49.0,62.0,0
3,2022-12-28,55,N,2023-01-02,N,0.0,0.0,2023-01-02,NaT,1A. PRIVATE,6166047,N,NaT,F,NaN,48.0,IV,25.0,10.0,53.0,6
4,2022-12-20,25,N,2023-01-02,N,0.0,1997.0,2022-12-31,NaT,1A. PRIVATE,6166102,N,NaT,M,NaN,55.0,IV,79.0,40.0,37.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387970,2012-09-12,52,N,2024-06-05,N,NaN,1960.0,2012-10-23,NaT,2A. SIF,6553137,N,NaT,M,NaN,NaN,I,NaN,NaN,NaN,5
387971,2024-05-22,59,N,2024-06-05,Y,0.0,1965.0,NaT,2024-05-28,3A. SELF PUBLIC,6553119,N,NaT,F,NaN,NaN,IV,NaN,NaN,NaN,1
387972,2024-05-06,45,N,2024-06-05,Y,0.0,1979.0,NaT,NaT,2A. SIF,6553542,N,NaT,M,NaN,NaN,IV,NaN,NaN,NaN,5
387973,2024-02-24,42,N,2024-06-05,Y,0.0,1981.0,NaT,2024-05-21,4A. SELF PRIVATE,6553455,N,NaT,M,NaN,NaN,IV,NaN,NaN,NaN,5


In [9]:
X_test[(~X_test['Carrier Type'].isna()) & (X_test['Industry Code'].isna())]

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Type,Claim Identifier,COVID-19 Indicator,First Hearing Date,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents
246,2022-12-21,59,Y,2023-01-03,Y,0.0,0.0,NaT,2023-01-30,4A. SELF PRIVATE,6166059,N,NaT,M,NaN,NaN,UK,NaN,NaN,NaN,0
273,2022-09-15,47,N,2023-01-03,N,0.0,1975.0,NaT,2022-12-27,3A. SELF PUBLIC,6165970,N,NaT,F,NaN,NaN,IV,NaN,NaN,NaN,5
335,2021-10-01,58,N,2023-01-03,N,0.0,1963.0,NaT,2022-11-04,3A. SELF PUBLIC,6166486,N,NaT,F,NaN,NaN,IV,NaN,NaN,NaN,4
346,2022-12-19,53,Y,2023-01-03,N,NaN,1969.0,NaT,NaT,4A. SELF PRIVATE,6165793,N,NaT,M,NaN,NaN,IV,NaN,NaN,NaN,3
393,2022-12-09,50,N,2023-01-03,N,0.0,1972.0,NaT,NaT,1A. PRIVATE,6166032,N,NaT,F,NaN,NaN,IV,NaN,NaN,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387970,2012-09-12,52,N,2024-06-05,N,NaN,1960.0,2012-10-23,NaT,2A. SIF,6553137,N,NaT,M,NaN,NaN,I,NaN,NaN,NaN,5
387971,2024-05-22,59,N,2024-06-05,Y,0.0,1965.0,NaT,2024-05-28,3A. SELF PUBLIC,6553119,N,NaT,F,NaN,NaN,IV,NaN,NaN,NaN,1
387972,2024-05-06,45,N,2024-06-05,Y,0.0,1979.0,NaT,NaT,2A. SIF,6553542,N,NaT,M,NaN,NaN,IV,NaN,NaN,NaN,5
387973,2024-02-24,42,N,2024-06-05,Y,0.0,1981.0,NaT,2024-05-21,4A. SELF PRIVATE,6553455,N,NaT,M,NaN,NaN,IV,NaN,NaN,NaN,5


### Replacing missing values for industry

In [10]:
# impute_with(df: pd.DataFrame, 
#   target_column: str, 
#   group_column = None,
#   unknown_values=['Unknown'],
#   reference_df=None,
#   metric = 'mode'):

# read in the X_train data
X_train = pd.read_csv('../project_data/train_preprocessed.csv', delimiter=',',dtype={'Zip Code': str})
X_train = X_train[~X_train['Industry Code'].isna()] # this is probably redundant 

# turn industry column to int 
X_test['Industry Code'].fillna('U')

# set index as column to not lose through imputation
X_test = X_test.reset_index()

# fill the casue of Injury Code with mode where it is missing with the most common value (mode) for Industry code in each cause of injury category
X_test_coI_not_missing_imputed =  impute_with(X_test[~X_test['WCIO Cause of Injury Code'].isna()],
                                               'Industry Code',
                                               'WCIO Cause of Injury Code',
                                                reference_df = X_train,
                                                unknown_values= ['U','X'],
                                                metric = 'mode')

# replace all the rows where X_test[X_test['WCIO Cause of Injury Code'].isna() & X_test['WCIO Cause of Injury Code'].isna()] with X_test_coI_not_missing_imputed
missing_condition_test = (~X_test['WCIO Cause of Injury Code'].isna()) & (X_test['Industry Code'].isna())
# Ensure that X_test_coI_not_missing_imputed has the same index structure or can align with X_test's missing rows
# X_test.loc[missing_condition_test] = X_test_coI_not_missing_imputed.loc[missing_condition_test]

# # impute the rest of the values with the overall mode value 
X_test['Industry Code'] = X_test['Industry Code'].fillna(X_test['Industry Code'].mode().values[0])

/Users/svengoerdes/Projects/Master/Machine_Learning/Machine_Learning_Project/deliverables/../helper_functions/helper_functions.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_column] = df[target_column].replace(unknown_values, pd.NA)
/Users/svengoerdes/Projects/Master/Machine_Learning/Machine_Learning_Project/deliverables/../helper_functions/helper_functions.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[target_column] = df.apply(
/Users/svengoerdes/Projects/Master/Machine_Learning/M

### Replace missing value for __Accident Date__ 

for this we will use the assembly date and calculate the median value of Accident date - assembly date and use that to obtain accident date


In [11]:
# for this we will use the assembly date and calculate the median value of 
# Accident date - assembly date and use that to obtain accident date

# Filter out rows where 'Accident Date' is missing
filtered_data = X_test[X_test['Accident Date'].notna()].copy()

# Calculate the difference in days between 'Assembly Date' and 'Accident Date'
filtered_data['Days Difference'] = (filtered_data['Assembly Date'] - filtered_data['Accident Date']).dt.days

# Calculate the median of the days difference
median_days_difference = filtered_data['Days Difference'].median()

del filtered_data 

# Replace missing Accident Date with assembly_date - median_distance
X_train['Accident Date'] = X_train.apply(
    lambda row: row['assembly_date'] - pd.Timedelta(days=median_days_difference) if pd.isna(row['Accident Date']) else row['Accident Date'],
    axis=1
    )

X_train.isna().sum()

Accident Date                             0
Age at Injury                             0
Alternative Dispute Resolution            0
Assembly Date                             0
Attorney/Representative                   0
Average Weekly Wage                       0
Birth Year                                0
C-2 Date                               9067
C-3 Date                             270351
Carrier Name                              0
Carrier Type                              0
County of Injury                          0
COVID-19 Indicator                        0
District Name                             0
First Hearing Date                   294696
Gender                                    0
IME-4 Count                               0
Industry Code                             0
Industry Code Description                 0
Medical Fee Region                        0
WCIO Cause of Injury Code                 0
WCIO Cause of Injury Description          0
WCIO Nature of Injury Code      

## Missing values for C-2 Date 
> As we saw before, it only makes sense for the C-2 Date to follow the Accident Date. Therefore, in cases where this is not true, we will assume that the Accident Date is correct and impute a new value for the C-2 Date. This new C-2 Date will be set to the Accident Date plus a certain number of days, corresponding to the median time difference (in days) between the Accident Date and C-2 Date, calculated only for X_train and cases where the Accident Date is earlier than the C-2 Date.

In [12]:
valid_dates_train = (X_train['Accident Date'] <= X_train['C-2 Date'])

valid_dates_test = (X_test['Accident Date'] <= X_test['C-2 Date'])

invalid_entries_train = X_train[~valid_dates_train & X_train['C-2 Date'].notna() & X_train['Accident Date'].notna()]
invalid_entries_test = X_test[~valid_dates_test & X_test['C-2 Date'].notna() & X_test['Accident Date'].notna()]

print("X_train:", invalid_entries_train.shape[0])
print("X_test:", invalid_entries_test.shape[0])

X_train: 0
X_test: 181


In [13]:
# get the rows where we dont have any missing c2 values 
# X_train_not_missing_c2
X_train_not_missing_c2 = X_train[~X_train['C-2 Date'].isna()]

# transform C2 date to datetime
X_train_not_missing_c2['C-2 Date'] = pd.to_datetime(X_train_not_missing_c2['C-2 Date'], format='%Y-%m-%d', errors='coerce') # errors='coerce' to handle NaT values
# transform Accident date to datetime
X_train_not_missing_c2['Accident Date'] = pd.to_datetime(X_train_not_missing_c2['Accident Date'], format='%Y-%m-%d', errors='coerce')

# Calculate the difference in days between 'C-2 Date' and 'Accident Date'
X_train_not_missing_c2['Date Difference'] = X_train_not_missing_c2.apply(
    lambda row: row['C-2 Date'] - row['Accident Date'] if row['C-2 Date'] > row['Accident Date'] else pd.NaT, axis=1
)

# In case we cant use Accidident date we will use the assembly date for the test data
# Calculate the difference in days between 'C-2 Date' and 'Accident Date'
X_train_not_missing_c2['Ass Date Difference'] = X_train_not_missing_c2.apply(
    lambda row: row['C-2 Date'] - row['Assembly Date'] if row['C-2 Date'] > row['Assembly Date'] else pd.NaT, axis=1
)

# Calculate the median of the differences in days for X_train
median_difference = X_train_not_missing_c2['Date Difference'].dropna().median().days
median_difference_ass = X_train_not_missing_c2['Assembly Date '].dropna().median().days

# Create the condition where Accident Date > C-2 Date and dates are not missing
condition_val = ((X_test['Accident Date'] > X_test['C-2 Date']) & (X_test['C-2 Date'].notna() & X_test['Accident Date'].notna())) 

# Impute C-2 Date
X_test.loc[condition_val, 'C-2 Date'] = X_test.loc[condition_val, 'Accident Date'] + pd.Timedelta(days=median_difference)

# Impute C-2 Date the same way for na values 
X_test['C-2 Date'].fillna(X_test['Accident Date'] + pd.Timedelta(days=median_difference), inplace=True)

X_test['C-2 Date'].fillna(X_test['Assembly Date'] + pd.Timedelta(days=median_difference_ass), inplace=True)

# Delete the newly created dataframe to free up memory
del X_train_not_missing_c2

/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_38658/3183029578.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_not_missing_c2['C-2 Date'] = pd.to_datetime(X_train_not_missing_c2['C-2 Date'], format='%Y-%m-%d', errors='coerce') # errors='coerce' to handle NaT values
/var/folders/nj/p20z0h9x3ssf6ll0_cdhh2yc0000gn/T/ipykernel_38658/3183029578.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_not_missing_c2['Accident Date'] = pd.to_datetime(X_train_not_missing_c2['Accident Dat

TypeError: '>' not supported between instances of 'Timestamp' and 'str'

In [ ]:
X_test.isna().sum()

index                                  0
Accident Date                       2444
Age at Injury                          0
Alternative Dispute Resolution         0
Assembly Date                          0
Attorney/Representative                0
Average Weekly Wage                19204
Birth Year                         19470
C-2 Date                             821
C-3 Date                          302759
Carrier Type                           0
Claim Identifier                       0
COVID-19 Indicator                     0
First Hearing Date                344947
Gender                                 0
IME-4 Count                       352726
Industry Code                          0
Medical Fee Region                     0
WCIO Cause of Injury Code          10348
WCIO Nature of Injury Code         10560
WCIO Part Of Body Code              9549
Number of Dependents                   0
dtype: int64

In [ ]:
X_test[~X_test['WCIO Cause of Injury Code'].isna()].isna().sum()

Accident Date                           867
Age at Injury                             0
Alternative Dispute Resolution            0
Assembly Date                             0
Attorney/Representative                   0
Average Weekly Wage                     526
Birth Year                              560
C-2 Date                               9133
C-3 Date                               4564
Carrier Name                              0
Carrier Type                              0
Claim Identifier                          0
County of Injury                          0
COVID-19 Indicator                        0
District Name                             0
First Hearing Date                     9539
Gender                                  177
IME-4 Count                           10195
Industry Code                          6007
Industry Code Description              6007
Medical Fee Region                        0
OIICS Nature of Injury Description    10348
WCIO Cause of Injury Code       

In [ ]:
X_test_coI_not_missing_imputed.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,Claim Identifier,County of Injury,COVID-19 Indicator,District Name,First Hearing Date,Gender,IME-4 Count,Industry Code,Industry Code Description,Medical Fee Region,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Number of Dependents
0,2022-12-24,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaN,INDEMNITY INSURANCE CO OF,1A. PRIVATE,6165911,BRONX,N,NYC,NaN,M,NaN,48.0,TRANSPORTATION AND WAREHOUSING,IV,NaN,31.0,"FALL, SLIP OR TRIP, NOC",10.0,CONTUSION,54.0,LOWER LEG,10466,1
1,2022-11-20,19,N,2023-01-02,N,NaN,2003.0,2023-01-02,NaN,A I U INSURANCE COMPANY,1A. PRIVATE,6166141,QUEENS,N,NYC,NaN,F,NaN,45.0,RETAIL TRADE,IV,NaN,75.0,FALLING OR FLYING OBJECT,10.0,CONTUSION,10.0,MULTIPLE HEAD INJURY,11691,1
2,2022-12-26,59,N,2023-01-02,N,0.0,1963.0,2022-12-31,NaN,AMGUARD INSURANCE COMPANY,1A. PRIVATE,6165907,WESTCHESTER,N,NYC,NaN,F,NaN,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,III,NaN,68.0,STATIONARY OBJECT,49.0,SPRAIN OR TEAR,62.0,BUTTOCKS,10604,0
3,2022-12-28,55,N,2023-01-02,N,0.0,0.0,2023-01-02,NaN,INDEMNITY INS. OF N AMERICA,1A. PRIVATE,6166047,QUEENS,N,NYC,NaN,F,NaN,48.0,TRANSPORTATION AND WAREHOUSING,IV,NaN,25.0,FROM DIFFERENT LEVEL (ELEVATION),10.0,CONTUSION,53.0,KNEE,11411,6
4,2022-12-20,25,N,2023-01-02,N,0.0,1997.0,2022-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,6166102,KINGS,N,NYC,NaN,M,NaN,55.0,MANAGEMENT OF COMPANIES AND ENTERPRISES,IV,NaN,79.0,OBJECT BEING LIFTED OR HANDLED,40.0,LACERATION,37.0,THUMB,11212,5
